In [1]:
from data_keeper import get_data_keeper

In [2]:
from testing import test_models_with_drugs

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV

In [4]:
get_data_keeper().get_possible_drugs()

[u'ETHA: Ethambutol ',
 u'ISON: Isoniazid ',
 u'RIFM: rifampicin ',
 u'RIFP: Rifapentine ',
 u'PYRA: Pyrazinamide ',
 u'STRE: Streptomycin ',
 u'CYCL: Cycloserine ',
 u'ETHI: Ethionamide/ Prothionamide ',
 u'PARA: Para-aminosalicyclic acid ',
 u'AMIK: Amikacin',
 u' Kanamycin ',
 u'CAPR: Capreomycin ',
 u'OFLO: Ofloxacin ']

In [5]:
from wrappers import GridSearchCVWrapper
from wrappers import XGBoostClassifierFeatureImportances as XGB
from wrappers import MatrixCleaningWrapper
from wrappers import SparseWrapper
from wrappers import ModelFeatureSelectionWrapper
from wrappers import ModelBasedFeatureImportanceGetter

In [6]:
from frn import FeatureRelevanceNetworkWrapper

In [7]:
def get_complete_linear_model():
    inner_model = LogisticRegressionCV(Cs=[10 ** i for i in xrange(-4, 4)], solver='liblinear')
    outer_model = GridSearchCV(inner_model, {'penalty': ['l1', 'l2']})
    return MatrixCleaningWrapper(SparseWrapper(outer_model))

In [ ]:
def get_complete_tree_based_model():
    cv_params = {'inner_model__inner_model__n_estimators': [1],#, 5, 10, 20, 50, 100], 
                 'feature_selection_threshold_coef': [0.1]}#, 1, 3, 10, 30, 100, 300]}
    return MatrixCleaningWrapper(FeatureRelevanceNetworkWrapper(SparseWrapper(XGB()), ModelBasedFeatureImportanceGetter(SparseWrapper(XGB(n_estimators=10)))))

In [ ]:
model = get_complete_tree_based_model()
test_models_with_drugs([('xgboost', model)], [u'ETHI: Ethionamide/ Prothionamide '], metrics=['accuracy', 'confusion matrix', 'feature count'])

cleaner (122, 2718)
{'f1401': 1, 'f1060': 1, 'f606': 3, 'f202': 3, 'f210': 2, 'f2193': 1, 'f160': 1, 'f2439': 3, 'f2168': 5, 'f970': 1, 'f458': 8, 'f2586': 2, 'f996': 8, 'f2161': 2, 'f504': 1}
<class 'scipy.sparse.csr.csr_matrix'>
cleaner (122, 2705)
{'f2157': 1, 'f198': 5, 'f2034': 1, 'f2411': 1, 'f1954': 1, 'f1499': 1, 'f95': 2, 'f994': 9, 'f995': 1, 'f2418': 5, 'f463': 10, 'f1389': 2}
<class 'scipy.sparse.csr.csr_matrix'>
cleaner (122, 2770)
{'f1417': 4, 'f630': 4, 'f2230': 2, 'f208': 1, 'f2479': 4, 'f1384': 1, 'f1011': 10, 'f99': 4, 'f476': 10}
<class 'scipy.sparse.csr.csr_matrix'>
cleaner (122, 2712)


In [ ]:
X = get_data_keeper().get_common_x()

In [ ]:
from testing import results_differ_p_value

In [ ]:
results_differ_p_value([0] * 100 + [1] * 32, [0] * 100 + [1] * 32, [0] * 99 + [1] * 32 + [0])